# ChatGPTは日本語の述語項構造解析をどのくらい行えるのか検証

## Open API keyの管理

Secret Manager APIで管理することにした。

設定の参考リンク
https://zenn.dev/hattan0523/articles/9be93149ac0754
https://zenn.dev/nananaoto/articles/43e3414fdb1da37b377e

In [ ]:
! pip install google-cloud-secret-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import secretmanager

def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

# こちらのPROJECT_IDはGoogle Cloud PlatformでのProject Idを指定する
PROJECT_ID = f"{your_project_id}"
# SECRET_NAMEも自分のものを指定すること
SECRET_NAME  = f"{your_secret_name}"
my_secret = access_secret(PROJECT_ID, SECRET_NAME)
print('yes')

yes


# データのロードと整形

KWDLCのWebコーパスをダウンロードして、Googleドライブに格納しています。
https://github.com/ku-nlp/KWDLC

## 前処理

- train/dev/testのIDの取得
- orgの取得、idと文のlistで保持
- knpフォルダ下の関係を取得

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# train/dev/testのIDの取得

import glob
import re
import os
import pprint as pp
# KWDLC/id/split_for_pas/

import json

# json_open = open('./sample_data/sample2.json', 'r')
json_open = open('/content/drive/My Drive/data/KWDLC/train_test_id.json', 'r')
train_test_id_dict = json.load(json_open)

# json_open = open('./sample_data/sample2.json', 'r')
json_open = open('/content/drive/My Drive/data/KWDLC/org.json', 'r')
org_dict = json.load(json_open)

# json_open = open('./sample_data/sample2.json', 'r')
json_open = open('/content/drive/My Drive/data/KWDLC/knp.json', 'r')
knp_dict = json.load(json_open)

# json_open = open('./sample_data/sample2.json', 'r')
json_open = open('/content/drive/My Drive/data/KWDLC/knp_pred.json', 'r')
knp_pred_dict = json.load(json_open)

files = glob.glob('/content/drive/My Drive/data/KWDLC/ok_result/*.json')

ok_doc_dict = {}
ok_count = 0
for file1 in files:
  json_open = open(file1, 'r')
  past_result_dict = json.load(json_open)
  for id1, tmp_dict in past_result_dict.items():
    if tmp_dict['status'] == 'ok':
      ok_doc_dict[id1] = 'ok'
      ok_count += 1

print([len(train_test_id_dict), len(org_dict), len(knp_dict), len(knp_pred_dict), len(ok_doc_dict), ok_count])

[3, 5127, 5127, 5127, 694, 1598]


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pprint as pp
# print(my_secret)
os.environ["OPENAI_API_KEY"] = my_secret

import openai

def ret_chatgpt_response_dict(org_temp_dict, pred_temp_dict, temperature=0):

  message_list = ret_prompt_text(org_temp_dict)

  # print(message_list)

  # 推論
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=temperature,
    messages=message_list
  )
  # print(response["choices"][0]["message"]["content"])
  return response
  # print(response["choices"][0]["message"]["content"])

def ret_role_dict(role_type, content):
  role_dict = {'role': role_type, 'content': content}
  return role_dict

def ret_input_sentence(org_tmp_dict):
  msg = ['以下の複数の文が与えられたとします。カッコの中が文の本文です。\n\n']
  sen_count = 1
  for sen_key, sentence in org_tmp_dict.items():
    rstr = f'文{sen_count} 「{sentence}」\n'
    msg.append(rstr)
    sen_count += 1
  msg.append('\n')
  return ''.join(msg)

def ret_prompt_text(org_tmp_dict):
  """version2
  """
  messages = []
  messages.append( ret_role_dict('system', 'あなたは日本語の文法に堪能なアシスタントです。日本語テキストの名詞や動詞といった品詞や主語や動詞が理解できます。') )
  sentences = ret_input_sentence(org_tmp_dict)
  messages.append( ret_role_dict('user', '以下に示す複数の文書はWebページの冒頭の3文です。まずこちらの文を読んでださい。') )
  messages.append( ret_role_dict('user', sentences) )

  messages.append( ret_role_dict('user', '各文の述語を抽出してください。述語は文に複数ある場合があります。連体修飾節の述語、サ変名詞、語尾の述語含めて、全て列挙してください。') )

  messages.append( ret_role_dict('user', '続いて、列挙した術語ごとに、述語の「主語（ガ格）」と「目的語（ヲ格）」と「斜格の要素（ニ格）」の候補を列挙してください'))
  messages.append( ret_role_dict('user', '注意点ですが、「主語（ガ格）」と「目的語（ヲ格）」と「斜格の要素（ニ格）」の候補は述語と同じ文中だけでなく、その文中では省略されている格要素も含めて列挙してください。 省略されている格要素には「著者」や「人一般」といった文に明示的に書かれていない要素も含めてください。') )
  # messages.append( ret_role_dict('user', '述語と主語と目的語の候補は、候補ごとに3つ組にしてください。結果は述語を1つの辞書とするjson line 形式でお願いします。') )
  messages.append( ret_role_dict('user', '最後に、述語、主語（ガ格）、目的語（ヲ格）、斜格の要素（ニ格）の候補の4つ組を作ってください。') )
  messages.append( ret_role_dict('user', 'そして、文の番号、述語、主語（ガ格）の候補、目的語（ヲ格）の候補、斜格の要素（ニ格）の5列を1つの行とするテーブル形式で出力してください。') )

  return messages

def ret_prompt_text1(org_tmp_dict):
  """ version1
  """
  messages = []
  messages.append( ret_role_dict('system', 'あなたは日本語の文法に堪能なアシスタントです。日本語テキストの名詞や動詞といった品詞や主語や動詞が理解できます。') )
  sentences = ret_input_sentence(org_tmp_dict)
  messages.append( ret_role_dict('user', '以下に示す複数の文書はWebページの冒頭の3文です。まずこちらの文を読んでださい。') )
  messages.append( ret_role_dict('user', sentences) )

  messages.append( ret_role_dict('user', '各文の述語を抽出してください。述語は文に複数ある場合があります。連体修飾節の述語、サ変名詞、語尾の述語含めて、全て列挙してください。') )

  messages.append( ret_role_dict('user', '続いて、列挙した術語ごとに、主語と目的語の候補を列挙してください'))
  messages.append( ret_role_dict('user', '主語と目的語の候補は述語と同じ文中だけでなく、文中では省略されているものや、筆者も含めて列挙してください。') )
  # messages.append( ret_role_dict('user', '述語と主語と目的語の候補は、候補ごとに3つ組にしてください。結果は述語を1つの辞書とするjson line 形式でお願いします。') )
  messages.append( ret_role_dict('user', '最後に、述語ごとの主語と目的語の候補の3つ組を作ってください。') )
  messages.append( ret_role_dict('user', '文の番号、述語、主語の候補、目的語の候補の4列を1つの行とするテーブル形式で出力してください。') )

  # messages.append( ret_role_dict('assistant', 'これら複数の文書はWebページの冒頭の3文です。') )
  # messages.append( ret_role_dict('user', '主語が文中にある言葉ならIN、主語が前文にある場合はINTER、主語が文中にない場合はOUTであることを示す列を加えてください。') )
  # messages.append( ret_role_dict('user', 'これら複数の文から、主語と述語の組を抽出して、json文字列にしてください。') )
  return messages

def evaluate_case_analysis(id):
  org_temp_dict = org_dict[id]
  pred_temp_dict = knp_pred_dict[id]
  # pp.pprint(org_temp_dict)
  # pp.pprint(pred_temp_dict)
  response_dict = {}
  try:
    response = ret_chatgpt_response_dict(org_temp_dict, pred_temp_dict, temperature=0)
    # print(response['choices'][0]['message']['content'])
    response_dict['status'] = 'ok'
    response_dict['response'] = response
  except Exception as e:
    response_dict['status'] = 'ng'
    response_dict['error_message'] = str(e)

  return response_dict

  # message_list = ret_prompt_text(org_temp_dict)
  # pp.pprint(message_list)
  # print([org_dict[id]])
  # print([knp_dict[id]])



In [ ]:
import logging

# ロガーを作成します。
logger = logging.getLogger(__name__)

# ロガーのレベルを設定します。
logger.setLevel(logging.INFO)

# ハンドラを作成します。
handler = logging.StreamHandler()

# ハンドラのフォーマットを設定します。
handler.setFormatter(logging.Formatter("%(asctime)s:%(levelname)s:%(message)s"))

# ロガーにハンドラを追加します。
logger.addHandler(handler)

# ログを出力します。
# logger.info("ログメッセージ")

In [ ]:
import time
import datetime
import os
import json
import time

def print_wait_log(rcount, success):
  if rcount % 100 == 0:
    # 現在の日時を取得
    current_datetime = datetime.datetime.now()
    # フォーマット指定して日時を表示
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    # logger.info(f'    {count}, {success}, {float(success)/float(count)}')
    print([formatted_datetime, rcount, float(success)/float(rcount)])

def save_result_json(result_dict, count):

  import json
  import datetime

  # 現在の日付を取得します。
  now = datetime.datetime.now()

  # ファイル名を作成します。
  filename = "result{date}.{c}.json".format(date=now.strftime("%Y%m%d_%H%M%S"), c=count)

  # JSON形式に変換する際に、インデントと文字コードを指定します
  json_data = json.dumps(result_dict, indent=4, ensure_ascii=False)

  BASE_DIR = '/content/drive/My Drive/data/'
  orgjsonfile = f'KWDLC/result/{filename}'
  filename = BASE_DIR + orgjsonfile
  # JSONデータをファイルに保存します
  with open(filename, 'w', encoding='utf-8') as f:
    f.write(json_data)


In [ ]:
import datetime
now = datetime.datetime.now()
count = 100

# ファイル名を作成します。
filename = "result{date}.{c}.json".format(date=now.strftime("%Y%m%d_%H:%M:%S"), c=count)
print(filename)

result20230527.100.json


In [ ]:
import time
import datetime
import os
import json
import time

# loop 回数
loop_count = 100

# APIリクエストの回数
request_count = 2

# 1分間の制限時間（秒）
limit_time = 60

# request発行回数
total_request_count = 0

# タイマーを初期化します。
timer = time.time()

rcount = 0
logging_count = 0
result_dict = {}
success = 0

phase_type = 'dev'
#for id in train_test_id_dict['test']:
for id in train_test_id_dict[phase_type]:
  if not id in ok_doc_dict:
    # API にリクエストを送信します。
    # print(["in", rcount, id])
    res = evaluate_case_analysis(id)
    # print(res)
    # res = {'status': 'ok'}
    res['phase'] = phase_type
    if res['status'] == 'ok':
      success += 1
    print(["out", res['status'], id])
    rcount += 1
    result_dict[id] = res

    if res['status'] == 'ok':
      success += 1

    if rcount == 1:
      #pp.pprint(res)
      #break
      pass

    if rcount % 100 == 0:
      save_result_json(result_dict, logging_count)
      logging_count += 1
    print_wait_log(rcount, success)

save_result_json(result_dict, logging_count)

['out', 'ok', 'w201106-0002000000']
['out', 'ok', 'w201106-0002000001']
['out', 'ok', 'w201106-0002000002']
['out', 'ok', 'w201106-0002000003']
['out', 'ok', 'w201106-0002000004']
['out', 'ok', 'w201106-0002000005']
['out', 'ok', 'w201106-0002000007']
['out', 'ok', 'w201106-0002000008']
['out', 'ok', 'w201106-0002000009']
['out', 'ok', 'w201106-0002000010']
['out', 'ok', 'w201106-0002000012']
['out', 'ok', 'w201106-0002000013']
['out', 'ok', 'w201106-0002000015']
['out', 'ok', 'w201106-0002000016']
['out', 'ok', 'w201106-0002000017']
['out', 'ok', 'w201106-0002000019']
['out', 'ok', 'w201106-0002000020']
['out', 'ok', 'w201106-0002000022']
['out', 'ok', 'w201106-0002000023']
['out', 'ok', 'w201106-0002000024']
['out', 'ok', 'w201106-0002000026']
['out', 'ok', 'w201106-0002000028']
['out', 'ok', 'w201106-0002000029']
['out', 'ok', 'w201106-0002000030']
['out', 'ok', 'w201106-0002000031']
['out', 'ok', 'w201106-0002000032']
['out', 'ok', 'w201106-0002000037']
['out', 'ok', 'w201106-00020

# 以下はOpenAIのAPIの呼び出し時のエラー回避のためのコード

一定時間の間隔を置いてAPI呼び出しを行うようにしたもの。OpenAIのアカウントの支払い設定を行えばGPT-3.5では必要がなくなった。

In [ ]:
import datetime
import os
import json
import time

def make_api_request(lcount, rcount, total_rcount):
  current_datetime = datetime.datetime.now()
  # フォーマット指定して日時を表示
  formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
  print([formatted_datetime, lcount, rcount, total_rcount])

# loop 回数
loop_count = 100

# APIリクエストの回数
request_count = 2

# 1分間の制限時間（秒）
limit_time = 60

# request発行回数
total_request_count = 0

for lcount in range(loop_count):
  # リクエストを実行するループ
  for rcount in range(request_count):
      # APIリクエストを実行
      make_api_request(lcount, rcount, total_request_count)
      total_request_count += 1
      if total_request_count == 10:
        break
      # 次のリクエストまで待機
      time.sleep(limit_time / request_count)

['2023-05-21 05:32:25', 0, 0, 0]
['2023-05-21 05:32:55', 0, 1, 1]
['2023-05-21 05:33:25', 1, 0, 2]
['2023-05-21 05:33:55', 1, 1, 3]
['2023-05-21 05:34:25', 2, 0, 4]
['2023-05-21 05:34:55', 2, 1, 5]
['2023-05-21 05:35:25', 3, 0, 6]
['2023-05-21 05:35:55', 3, 1, 7]
['2023-05-21 05:36:25', 4, 0, 8]
['2023-05-21 05:36:55', 4, 1, 9]
['2023-05-21 05:36:55', 5, 0, 10]
['2023-05-21 05:37:25', 5, 1, 11]
['2023-05-21 05:37:55', 6, 0, 12]
['2023-05-21 05:38:25', 6, 1, 13]
['2023-05-21 05:38:55', 7, 0, 14]
['2023-05-21 05:39:25', 7, 1, 15]
['2023-05-21 05:39:55', 8, 0, 16]
['2023-05-21 05:40:25', 8, 1, 17]
['2023-05-21 05:40:55', 9, 0, 18]
['2023-05-21 05:41:25', 9, 1, 19]
['2023-05-21 05:41:55', 10, 0, 20]
['2023-05-21 05:42:25', 10, 1, 21]
['2023-05-21 05:42:55', 11, 0, 22]
['2023-05-21 05:43:25', 11, 1, 23]
['2023-05-21 05:43:55', 12, 0, 24]
['2023-05-21 05:44:25', 12, 1, 25]
['2023-05-21 05:44:55', 13, 0, 26]
['2023-05-21 05:45:25', 13, 1, 27]
['2023-05-21 05:45:55', 14, 0, 28]
['2023-05-21 05

KeyboardInterrupt: ignored

In [ ]:
import requests
import time

def make_api_request2(lcount, rcount):
  current_datetime = datetime.datetime.now()
  # フォーマット指定して日時を表示
  formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
  print([formatted_datetime, lcount, rcount])


# loop 回数
loop_count = 100

# APIリクエストの回数
request_count = 2

# 1分間の制限時間（秒）
limit_time = 60

# request発行回数
total_request_count = 0

# タイマーを初期化します。
timer = time.time()

for lcount in range(loop_count):
  # リクエストを送信します。
  while True:
      # 現在時刻を取得します。
      now = time.time()

      # 前回の呼び出しから 1 分以上経過している場合、リクエストを送信します。
      if now - timer >= 30:
          # API にリクエストを送信します。
          print("in")
          make_api_request2(lcount, rcount)
          rcount += 1
          if rcount == 10:
            break
          # タイマーを更新します。
          timer = now

      print("wait")
      # 1 秒間待機します。
      time.sleep(1)

in
['2023-05-21 05:59:20', 0, 2]
in
['2023-05-21 05:59:50', 0, 3]
in
['2023-05-21 06:00:20', 0, 4]
in
['2023-05-21 06:00:50', 0, 5]
in
['2023-05-21 06:01:20', 0, 6]
in
['2023-05-21 06:01:50', 0, 7]
in
['2023-05-21 06:02:20', 0, 8]
in
['2023-05-21 06:02:50', 0, 9]
in
['2023-05-21 06:02:50', 1, 10]
in
['2023-05-21 06:03:20', 1, 11]
in
['2023-05-21 06:03:50', 1, 12]
in
['2023-05-21 06:04:20', 1, 13]
in
['2023-05-21 06:04:50', 1, 14]
in
['2023-05-21 06:05:20', 1, 15]
in
['2023-05-21 06:05:50', 1, 16]
in
['2023-05-21 06:06:20', 1, 17]
in
['2023-05-21 06:06:50', 1, 18]
in
['2023-05-21 06:07:20', 1, 19]
in
['2023-05-21 06:07:50', 1, 20]
in
['2023-05-21 06:08:21', 1, 21]


KeyboardInterrupt: ignored